<a href="https://colab.research.google.com/github/praveenkumarprabakar18/Classnotes/blob/main/Scripts/Lab_test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve


In [ ]:
data_path = "https://raw.githubusercontent.com/Bits-Deep-Analytics/Predictive_analytics/dev/Data/PIMA/diabetes.csv"
dataset = pd.read_csv(data_path)
display(dataset.head(10))
dataset.shape

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.describe()

In [ ]:
data_with_zeros = dataset[
    (dataset['Glucose'] == 0)
    | (dataset['BMI'] == 0)
    | (dataset['Insulin'] == 0)
    | (dataset['BloodPressure'] == 0)
]
print('N of examples with incomplete data = {}'.format(len(data_with_zeros)))

In [ ]:
#columns=['BMI', 'SkinThickness', 'Insulin', 'Glucose', 'BloodPressure' ]
#for i in range(0, len(columns)):

 # dataset[columns[i]].replace(0, np.nan, inplace=True)
  #dataset.dropna(subset=columns[i], inplace=True)
#print(dataset.shape)
  
#Removing Zeros is almost removing half of the rows in the orginal dataset, the newshape is (392,9), and the model 

In [ ]:
dataset.corr()

In [ ]:
plt.figure(figsize=(12,9))
s=sns.heatmap(dataset.corr(), annot=True, cmap='RdBu', vmin=-1, vmax=1)
s.set_yticklabels(s.get_yticklabels(), rotation=0, fontsize=12)
s.set_xticklabels(s.get_xticklabels(), rotation=90, fontsize=12)
plt.title("Correlation Heat Map")
plt.show()

In [ ]:
scalar=StandardScaler()
scaled_dataset=scalar.fit_transform(dataset)
scaled_dataset


In [ ]:

pca= PCA()
pca.fit(scaled_dataset)
#The below code shows how much variance is explained by each component
pca.explained_variance_ratio_

In [ ]:
plt.figure(figsize=(12,9))
plt.plot(range(1,10), pca.explained_variance_ratio_.cumsum(), marker='o', linestyle="--" )
plt.title("Explained variance by Components")
plt.xlabel("Number of components")
plt.ylabel("Cumulative Explained variance")
#6 components explain 90% of the variance

In [ ]:
x=dataset.loc[:,dataset.columns!='Outcome']
y=dataset.loc[:,dataset.columns=='Outcome']
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.2, random_state=0)

In [ ]:
steps=[('Scaling', StandardScaler()), ('Reducing_dimension', PCA(n_components=0.90)), ('Modelling', LogisticRegression())]
pipeline=Pipeline(steps)
pipeline.fit(x_train,np.ravel(y_train))
y_pred1=pipeline.predict(x_test)
acc=metrics.accuracy_score(y_true=y_test, y_pred=y_pred1)
c_matrix=metrics.confusion_matrix(y_true=y_test, y_pred=y_pred1)
print("Accuracy is", acc)
print(c_matrix)
                   

In [ ]:
precision1=round(metrics.precision_score(y_true=y_test, y_pred=y_pred1)*100,2)
recall1=round(metrics.recall_score(y_true=y_test, y_pred=y_pred1)*100,2)
print("Precision is ", precision1)
print("Recall is", recall1)


In [ ]:
#After checking different possibilities, we found n_neighbours=7, and minkowski distance of p=2
steps=[('Scaling', StandardScaler()), ('Reducing_dimension', PCA(n_components=0.90)), ('Modelling', KNeighborsClassifier(n_neighbors= 7, metric='minkowski', p=2 ))]
pipeline=Pipeline(steps)
pipeline.fit(x_train,np.ravel(y_train))
y_pred2=pipeline.predict(x_test)
acc=metrics.accuracy_score(y_true=y_test, y_pred=y_pred2)
c_matrix=metrics.confusion_matrix(y_true=y_test, y_pred=y_pred2)
print("Accuracy is", acc)
print(c_matrix)

In [ ]:
precision2=round(metrics.precision_score(y_true=y_test, y_pred=y_pred2)*100,2)
recall2=round(metrics.recall_score(y_true=y_test, y_pred=y_pred2)*100,2)
print("Precision is ", precision2)
print("Recall is", recall2)

In [ ]:
l=[]
for i in range(0,9):
  x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
  classifier=KNeighborsClassifier(n_neighbors= 5, metric='minkowski', p=2 )
  classifier.fit(x_train, y_train)
  y_prediction = classifier.predict(x_test)
  cm=confusion_matrix(y_test, y_prediction)
  acc=accuracy_score(y_test, y_prediction)
  l.append(acc)

avg=0
sum=0
for i in range(0,9):
  sum+=l[i]
avg=sum/len(l)
print(avg)
  

In [ ]:
auc_lr = roc_auc_score(y_test, y_pred1)
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_pred1)

auc_knn = roc_auc_score(y_test, y_pred2)
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, y_pred2)

plt.figure(figsize=(12, 7))
plt.plot(fpr_lr, tpr_lr, label=f'AUC (Logistic Regression) = {auc_lr:.2f}')
plt.plot(fpr_knn, tpr_knn, label=f'AUC (KNN) = {auc_knn:.2f}')

plt.title('ROC Curve', size=20)
plt.xlabel('False Positive Rate', size=14)
plt.ylabel('True Positive Rate', size=14)
plt.legend();
roc_auc_score(y_test,y_pred1)
roc_auc_score(y_test,y_pred2)

In [ ]:
from sklearn.model_selection import KFold
size= dataset.size
# create the range of dataset
rn = range(size)
# to demonstrate how the data are split, we will create 3 and 5 folds.
# KFold function has to be applied on the data and it returns an location (index) of the train and test samples.
kf10 = KFold(n_splits=10, shuffle=False)
for train_index, test_index in kf10.split(rn):
    print(train_index, test_index)